In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import time

dataframe = pd.read_csv("C:/Users/38164/Master studije/Data Mining/05 DM Asocijativna analiza/retail_dataset.csv", sep=',')
dataframe

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
310,Bread,Eggs,Cheese,NaN,NaN,NaN,NaN
311,Meat,Milk,Pencil,NaN,NaN,NaN,NaN
312,Bread,Cheese,Eggs,Meat,Pencil,Diaper,Wine
313,Meat,Cheese,NaN,NaN,NaN,NaN,NaN


In [2]:
dataframe.info

<bound method DataFrame.info of           0       1       2       3       4       5       6
0     Bread    Wine    Eggs    Meat  Cheese  Pencil  Diaper
1     Bread  Cheese    Meat  Diaper    Wine    Milk  Pencil
2    Cheese    Meat    Eggs    Milk    Wine     NaN     NaN
3    Cheese    Meat    Eggs    Milk    Wine     NaN     NaN
4      Meat  Pencil    Wine     NaN     NaN     NaN     NaN
..      ...     ...     ...     ...     ...     ...     ...
310   Bread    Eggs  Cheese     NaN     NaN     NaN     NaN
311    Meat    Milk  Pencil     NaN     NaN     NaN     NaN
312   Bread  Cheese    Eggs    Meat  Pencil  Diaper    Wine
313    Meat  Cheese     NaN     NaN     NaN     NaN     NaN
314    Eggs    Wine   Bagel   Bread    Meat     NaN     NaN

[315 rows x 7 columns]>

In [3]:
dataframe.isnull().sum()

0      0
1     30
2     70
3    128
4    182
5    244
6    274
dtype: int64

In [4]:
items = set()
for col in dataframe:
    items.update(dataframe[col].unique())
items.remove(np.nan)
print(items)

{'Pencil', 'Eggs', 'Meat', 'Wine', 'Bagel', 'Bread', 'Cheese', 'Diaper', 'Milk'}


In [5]:
for feature_index in dataframe.columns:
    dataframe[feature_index]=dataframe[feature_index].replace(np.nan, 0)

In [6]:
dataframe

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,0,0
3,Cheese,Meat,Eggs,Milk,Wine,0,0
4,Meat,Pencil,Wine,0,0,0,0
...,...,...,...,...,...,...,...
310,Bread,Eggs,Cheese,0,0,0,0
311,Meat,Milk,Pencil,0,0,0,0
312,Bread,Cheese,Eggs,Meat,Pencil,Diaper,Wine
313,Meat,Cheese,0,0,0,0,0


# Algoritam:

**Transformacija dataset-a u listu listi**

In [7]:
df=dataframe.to_numpy()
df_list=[]
for item in df:
    item=item.tolist()
    zero_indices=[]
    for value in range(0, len(item)):
        if item[value]==0:
            zero_indices.append(value)
    res=[]
    idx=0
    for ele in item:
        if idx not in zero_indices:
            res.append(ele)
        idx+=1
    df_list.append(res)

In [8]:
df_list

[['Bread', 'Wine', 'Eggs', 'Meat', 'Cheese', 'Pencil', 'Diaper'],
 ['Bread', 'Cheese', 'Meat', 'Diaper', 'Wine', 'Milk', 'Pencil'],
 ['Cheese', 'Meat', 'Eggs', 'Milk', 'Wine'],
 ['Cheese', 'Meat', 'Eggs', 'Milk', 'Wine'],
 ['Meat', 'Pencil', 'Wine'],
 ['Eggs', 'Bread', 'Wine', 'Pencil', 'Milk', 'Diaper', 'Bagel'],
 ['Wine', 'Pencil', 'Eggs', 'Cheese'],
 ['Bagel', 'Bread', 'Milk', 'Pencil', 'Diaper'],
 ['Bread', 'Diaper', 'Cheese', 'Milk', 'Wine', 'Eggs'],
 ['Bagel', 'Wine', 'Diaper', 'Meat', 'Pencil', 'Eggs', 'Cheese'],
 ['Cheese', 'Meat', 'Eggs', 'Milk', 'Wine'],
 ['Bagel', 'Eggs', 'Meat', 'Bread', 'Diaper', 'Wine', 'Milk'],
 ['Bread', 'Diaper', 'Pencil', 'Bagel', 'Meat'],
 ['Bagel', 'Cheese', 'Milk', 'Meat'],
 ['Bread'],
 ['Pencil', 'Diaper', 'Bagel'],
 ['Meat', 'Bagel', 'Bread'],
 ['Bread', 'Bagel', 'Milk'],
 ['Diaper'],
 ['Bagel', 'Cheese', 'Meat', 'Bread', 'Diaper', 'Eggs'],
 ['Meat', 'Pencil', 'Cheese', 'Bread'],
 ['Cheese', 'Eggs', 'Wine', 'Bread', 'Milk', 'Pencil', 'Meat'],
 ['

In [9]:
items=list(items)

**Implementacija Apriori algoritma**

In [10]:
#funkcija za generisanje frekventnong skupa stavki
def generate_frequent_itemset(list_of_transactions, minsup): 
    freq=[]
    non_freq=[]
    num=0
    n=len(list_of_transactions)
    for item in items:
        num=0
        for row in list_of_transactions:
            if item in row:
                num+=1
        if num/n>minsup:
            freq.append(item)
    max_k=len(freq)
    k=2
    while k<max_k:
        potential_candidates=itertools.combinations(items, k)
        potential_candidates=list(potential_candidates)
        new_candidates=[]
        non_freq_candidates=[]
        if len(non_freq)>0:
            for item_1 in potential_candidates:
                is_non_freq=0
                for item_2 in non_freq:
                    if set(item_2).issubset(item_1):
                        non_freq_candidates.append(item_1)
                        is_non_freq=1
                        continue
                if is_non_freq==0:
                    new_candidates.append(item_1)
        else:
            new_candidates=potential_candidates
        non_freq+=non_freq_candidates
        for item in new_candidates:
            num=0
            for row in df_list:
                if set(item).issubset(row):
                    num+=1
            if num/n>minsup:
                freq.append(item)
            else:
                non_freq.append(item)
        k+=1
    return freq

In [132]:
def generate_rules(freq_itemset,minsup, minconf):
    
    st = time.time()
    
    potential_rules=[]
    freq=generate_frequent_itemset(freq_itemset, minsup)
    for item in freq:
        if (len(set(item))>1)&(type(item)==tuple):
            potential_rules.append(item)
    rules={}        
    all_possible_y=[]
    all_possible_x=[]
    key=0
    for potential_rule in potential_rules:
        all_possible_y=[]
        all_possible_x=[]
        low_conf_rules=[]
        for i in range(1, len(potential_rule)):
            potential_y=itertools.combinations(potential_rule, i)
            all_possible_y+=list(potential_y)    
            for item in all_possible_y:
                is_low_conf=0
                #Deo koda za odsecanje pravila sa niskom pouzadnoscu
                if len(low_conf_rules)>0:
                    for low_conf_rule in low_conf_rules:
                        if set(low_conf_rule).issubset(item):
                            is_low_conf=1
                if is_low_conf==1:
                    continue
                #Deo koda za odsecanje pravila sa niskom pouzadnoscu
                x=set(potential_rule).difference(item)
                x_and_y=set(item).union(x)
                num_xy=0
                num_x=0
                support_xy=0
                support_x=0
                for row in freq_itemset:
                    if x_and_y.issubset(set(row)):
                        num_xy+=1
                    if x.issubset(set(row)):
                        num_x+=1
                support_xy=num_xy/len(freq_itemset)
                support_x=num_x/len(freq_itemset)
                if support_xy/support_x>minconf:
                    rule={'x':list(x), 'y':list(item), 'conf':support_xy/support_x}
                    rules['tid'+str(key)]=rule
                    key+=1
                else:
                    low_conf_rules.append(list(item))
                    
    et = time.time()
    elapsed_time = et - st
    print('Vreme izvrsenja:', elapsed_time, 'seconds')
    return rules

Vreme izvrsenja sa ranijem odsecanjem asocijativnih pravila sa niskom pouzdanoscu:

In [95]:
my_rules=generate_rules(df_list, 0.005, 0.4)

Vreme izvrsenja: 1.3796818256378174 seconds


Vreme izvrsenja bez ranijeg odsecanja asocijativnih pravila sa niskom pouzdanoscu:

In [131]:
my_rules=generate_rules(df_list, 0.005, 0.4)

Vreme izvrsenja: 5.050975799560547 seconds


Primer pravila:

In [134]:
asoc_rules=generate_rules(df_list, 0.1, 0.4)
asoc_rules

Vreme izvrsenja: 0.4907100200653076 seconds


{'tid0': {'x': ['Pencil'], 'y': ['Eggs'], 'conf': 0.45614035087719296},
 'tid1': {'x': ['Pencil'], 'y': ['Meat'], 'conf': 0.4912280701754386},
 'tid2': {'x': ['Wine'], 'y': ['Pencil'], 'conf': 0.4565217391304348},
 'tid3': {'x': ['Pencil'], 'y': ['Wine'], 'conf': 0.5526315789473685},
 'tid4': {'x': ['Pencil'], 'y': ['Bagel'], 'conf': 0.43859649122807015},
 'tid5': {'x': ['Pencil'], 'y': ['Bread'], 'conf': 0.5526315789473685},
 'tid6': {'x': ['Pencil'], 'y': ['Cheese'], 'conf': 0.5526315789473685},
 'tid7': {'x': ['Diaper'], 'y': ['Pencil'], 'conf': 0.421875},
 'tid8': {'x': ['Pencil'], 'y': ['Diaper'], 'conf': 0.4736842105263158},
 'tid9': {'x': ['Pencil'], 'y': ['Milk'], 'conf': 0.4736842105263158},
 'tid10': {'x': ['Meat'], 'y': ['Eggs'], 'conf': 0.56},
 'tid11': {'x': ['Eggs'], 'y': ['Meat'], 'conf': 0.6086956521739131},
 'tid12': {'x': ['Wine'], 'y': ['Eggs'], 'conf': 0.5507246376811594},
 'tid13': {'x': ['Eggs'], 'y': ['Wine'], 'conf': 0.5507246376811594},
 'tid14': {'x': ['Eggs']